In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from patsy import dmatrices

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [5]:
# pbp_data = pd.read_csv("./results/base-epa.csv", encoding="utf-8")
pbp_data = pd.DataFrame()
years = range(2014,2020)
for i in years:
    url = f"https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_{i}.parquet"
    print(f"Pulling url {url}")
    i_data = pd.read_parquet(url, engine='pyarrow')

    #sort=True eliminates a warning and alphabetically sorts columns
    pbp_data = pbp_data.append(i_data, sort=True)

#Give each row a unique index
pbp_data.reset_index(drop=True, inplace=True)
pbp_data.head()

Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2014.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2015.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2016.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2017.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2018.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2019.parquet


,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [6]:
game_data = pd.DataFrame()

# downloaded files from collegefootballdata.com
def retrieveCfbDataFile(endpoint, year):
    return pd.read_csv(f"data/{endpoint}/{year}.csv", encoding='latin-1')

for i in range(2012, 2020):
    gm = retrieveCfbDataFile('games',i)
    gm['game_id'] = gm['id']
    gm['year'] = i
    gm['winner'] = np.where(gm.home_points > gm.away_points, gm.home_team, gm.away_team)
    game_data = game_data.append(gm, sort=False)

print(f"Total Games: {len(game_data)}")

Total Games: 6644


In [7]:
pbp_data = pd.merge(pbp_data, game_data[['game_id','winner']], left_on="game_id", right_on="game_id")
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [8]:
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [29]:
pbp_data['win_indicator'] = pbp_data.pos_team == pbp_data.winner
pbp_data.half = pbp_data.half.astype(int)
# len(pbp_data[pbp_data.off_timeouts_rem_before.isna()])
pbp_data.loc[pbp_data.off_timeouts_rem_before.isna(), "off_timeouts_rem_before"] = 0
# len(pbp_data[pbp_data.def_timeouts_rem_before.isna()])
pbp_data.loc[pbp_data.def_timeouts_rem_before.isna(), "def_timeouts_rem_before"] = 0

# pbp_data.time_remaining = np.where(pbp_data.half == 1, 1800 + pbp_data.time_remaining, pbp_data.time_remaining)
# pbp_data['def_EPA'] = -1 * pbp_data.EPA
# pbp_data['home_EPA'] = np.where(pbp_data.offense == pbp_data.home_team, pbp_data.EPA, -1 * pbp_data.EPA)
# pbp_data['away_EPA'] = -1 * pbp_data.home_EPA
# pbp_data['score_diff'] = pbp_data.offense_score - pbp_data.defense_score

# https://zacharyst.com/2019/03/06/group_by-mutate-using-pandas/
# def determine_turnover_lag(data):
#     data['turnover_vec_lag'] = data.sort_values(by="play_id").turnover_indicator.shift(1)
#     return data

# pbp_data = pbp_data.groupby('game_id').apply(determine_turnover_lag)
# pbp_data.loc[pbp_data.turnover_vec_lag.isna(), 'turnover_vec_lag'] = 0
# pbp_data.head()

In [30]:
defense_score_vec = [
    "Blocked Punt Touchdown",
    "Blocked Field Goal Touchdown",
    "Missed Field Goal Return Touchdown",
    "Punt Return Touchdown",
    "Fumble Recovery (Opponent) Touchdown",    
    "Fumble Return Touchdown",
    "Fumble Return Touchdown Touchdown",
    "Defensive 2pt Conversion",
    "Safety",
    "Sack Touchdown",    
    "Interception Return Touchdown",
    "Pass Interception Return Touchdown",
    "Uncategorized Touchdown"
]
# pbp_data['def_td_play'] = pbp_data.play_type.isin(defense_score_vec)

# def determine_def_td_play_lag(data):
#     data['def_td_play_lag'] = data.sort_values(by="play_id").def_td_play.shift(1)
#     return data

# pbp_data = pbp_data.groupby('game_id').apply(determine_def_td_play_lag)
# pbp_data['play_after_turnover'] = (pbp_data.def_td_play_lag == True) & (pbp_data.turnover_vec_lag == True)


# def determine_offense_lag(data):
#     data['offense_lag'] = data.sort_values(by="play_id").offense.shift(1)
#     return data

# pbp_data = pbp_data.groupby('game_id').apply(determine_offense_lag)
# pbp_data.loc[pbp_data.offense_lag.isna(),'offense_lag'] = pbp_data.offense

# def determine_offense_lead(data, val):
#     data[f'offense_lead{"" if abs(val) == 1 else abs(val)}'] = data.sort_values(by="play_id").offense.shift(val)
#     return data

# pbp_data = pbp_data.groupby('game_id').apply(lambda x: determine_offense_lead(x, -1))
# pbp_data = pbp_data.groupby('game_id').apply(lambda x: determine_offense_lead(x, -2))

# def determine_lag_score_diff(data):
#     data['lag_score_diff'] = data.sort_values(by="play_id").score_diff.shift(1)
#     return data

# pbp_data = pbp_data.groupby('game_id').apply(determine_lag_score_diff)
# pbp_data.loc[pbp_data.lag_score_diff.isna(), 'lag_score_diff'] = 0

# pbp_data['score_pts'] = np.where(pbp_data.offense_lag == pbp_data.offense, pbp_data.score_diff - pbp_data.lag_score_diff, pbp_data.score_diff + pbp_data.lag_score_diff) 
# pbp_data['score_diff_start'] = np.where(pbp_data.offense_lag == pbp_data.offense, pbp_data.lag_score_diff, -1 * pbp_data.lag_score_diff)
# pbp_data['ExpScoreDiff'] = pbp_data.score_diff_start + pbp_data.EP_start
# pbp_data['ExpScoreDiff_TimeRatio'] = (pbp_data['ExpScoreDiff'] / (pbp_data.time_remaining + 1))

# pbp_data.head()

In [31]:
wpa_formula = "win_indicator ~ cr(ExpScoreDiff,df=2) + cr(adj_TimeSecsRem,df=2) + cr(ExpScoreDiff_Time_Ratio,df=2) + Under_two*off_timeouts_rem_before*C(half) + Under_two*def_timeouts_rem_before*C(half)"
dtrain_y, dtrain_x = dmatrices(wpa_formula, pbp_data, return_type='dataframe')
dtrain_x.columns

PatsyError: Error evaluating factor: ValueError: object arrays are not supported
    win_indicator ~ cr(ExpScoreDiff,df=2) + cr(adj_TimeSecsRem,df=2) + cr(ExpScoreDiff_Time_Ratio,df=2) + Under_two*off_timeouts_rem_before*C(half) + Under_two*def_timeouts_rem_before*C(half)
                                            ^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
dtrain = xgb.DMatrix(X, weight=regression_df.Total_W_Scaled, label=y.next_drive_point)

xgb_model = xgb.train(params, dtrain, num_boost_round=nrounds, verbose_eval=2)
# save for debug
xgb_model.dump_model('xgb_dump.json', with_stats=True, dump_format='json')